In [2]:
import pandas as pd
import os
import threading
from sklearn.ensemble import GradientBoostingRegressor
import altair as alt
import numpy as np
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# https://www.kaggle.com/datasets

a_i_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv"
with_slash = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\'"

d_i_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\d-i"
with_slash4 = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\d-i\'"

j_p_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p"
with_slash2 = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p\'"

q_z_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p\q-z"
with_slash3 = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p\q-z\'"

df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
dataframes = []

In [4]:
# function to combine two datasets
def combine(dat1, dat2):
    to_combine = [dat1, dat2]
    return pd.concat(to_combine)


def get_company_name(id):
    msft = yf.Ticker(id)

    company_name = msft.info['longName']
    return company_name

In [5]:
# function to loop through and open files
def files_to_dataframe(directory_path, with_slash, df):
    for filename in os.listdir(directory_path):
        if filename == 'j-p'or filename == 'q-z' or filename == 'd-i':
            continue
        directory = with_slash[:-1]
        path = directory + filename
        with open(path, 'r') as file:
            dat = pd.read_csv(file)
        dat['company_id'] = filename[:len(filename)-4]
        df = combine(df, dat)
    dataframes.append(df)

# loops through the folder broken up into four threads that run concurently 
thread1 = threading.Thread(target=files_to_dataframe, args=(a_i_directory_path, with_slash, df))
thread2 = threading.Thread(target=files_to_dataframe, args=(j_p_directory_path, with_slash2, df2))
thread3 = threading.Thread(target=files_to_dataframe, args=(q_z_directory_path, with_slash3, df3))
thread4 = threading.Thread(target=files_to_dataframe, args=(d_i_directory_path, with_slash4, df4))

thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread1.join()
thread2.join()
thread3.join()
thread4.join()

# combining the dataframes from each of the four threads
df = combine(dataframes[0], dataframes[1])
df2 = combine(dataframes[2], dataframes[3])
df = combine(df, df2)
df.dropna(inplace=True)
print(len(df))

# df['company'] = df.apply(get_company_name, axis=1)
# df.drop(columns=['company_id'])

3258423


## What is the biggest difference in the low and high?

In [ ]:
# ==================================================== DATA ANALYSIS =========================================

df['difference'] = df.apply(lambda x: x.High - x.Low, axis=1)

max = df["difference"].max()
df.set_index("difference", inplace=True)

high = df.loc[max,'High']
low = df.loc[max,'Low']
company = get_company_name(df.loc[max,'company_id'])
date = df.loc[max, 'Date']

print(f"The biggest difference in high to low is {max} from {company}")
print(f'The high was {high:.2f} and the low was {low:.2f}')
print(f'The date was {date}')

df.reset_index(inplace=True)

The biggest difference in high to low is 53156.0 from Berkshire Hathaway Inc.
The high was 468795.00 and the low was 415639.00
The date was 27-10-2022


## What is the largest difference by percentage?

In [ ]:
df['percent_difference'] = df.apply(lambda x: (x.difference / x.High) * 100, axis=1)
max = df['percent_difference'].max()

df.set_index("percent_difference", inplace=True)
company = df.loc[max, 'company_id']
high = df.loc[max, 'High']
low = df.loc[max, 'Low']
date = df.loc[max, 'Date']

print(f"The biggest difference by percent from high to low is {max} from {get_company_name(company)}")
print(f'The high was {high:.2f} and the low was {low:.4f}')
print(f'The date was {date}')

df.reset_index(inplace=True)
# print(df.head())

The biggest difference by percent from high to low is 99.98492462247178 from Formcap Corp.
The high was 1.99 and the low was 0.0003
The date was 23-11-2020


## What is the oldest and newest date?

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['yr'] = pd.DatetimeIndex(df['Date']).year
oldest = df['Date'].min()
newest = df['Date'].max()
print(f' The oldest date is {oldest}')
print(f'The newest date is {newest}')
print(df.head())

C:\Users\Blake Dennett\AppData\Local\Temp\ipykernel_772\4159101142.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Date'] = pd.to_datetime(df['Date'])


 The oldest date is 1970-01-04 00:00:00
The newest date is 2022-12-12 00:00:00
   percent_difference  difference       Date        Low       Open     Volume  \
0            5.540352    3.988998 2015-02-01  68.010002  69.000000  4218700.0   
1            5.946712    4.240005 2015-05-01  67.059998  71.230003  4534700.0   
2            7.309895    4.969997 2015-06-01  63.020000  67.930000  3749400.0   
3            4.868190    3.250004 2015-07-01  63.509998  65.010002  2122600.0   
4            3.673418    2.560005 2015-08-01  67.129997  67.949997  2510300.0   

        High      Close  Adjusted Close company_id    yr  
0  71.999001  70.400002       70.400002       QRVO  2015  
1  71.300003  67.629997       67.629997       QRVO  2015  
2  67.989998  64.669998       64.669998       QRVO  2015  
3  66.760002  66.650002       66.650002       QRVO  2015  
4  69.690002  67.690002       67.690002       QRVO  2015  


# Machine Learning to Guess the Close

## Vanilla Tree

In [ ]:
df.rename(columns={'difference':' difference', 'percent_difference':' percent_difference'}, inplace=True)
X = df[[' difference', ' percent_difference', 'High', 'Low', 'Open', 'Volume']]
y = df['Close']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)
reg = GradientBoostingRegressor(random_state=25)
reg.fit(x_train, y_train)
y_predictions = reg.predict(x_test)
r2 = r2_score(y_test, y_predictions)
rmse = mean_squared_error(y_test, y_predictions)
print(f'The r^2 is: {r2} and the RMSE is: {rmse}')

The r^2 is: 0.9999138806589019 and the RMSE is: 6645.893880489394


## Visualizations

In [9]:

alt.data_transformers.disable_max_rows()
print(df.head())
id_df = df.query('Close < 50')
id_df.head()
len(id_df)

         Date        Low       Open     Volume       High      Close  \
0  02-01-2015  68.010002  69.000000  4218700.0  71.999001  70.400002   
1  05-01-2015  67.059998  71.230003  4534700.0  71.300003  67.629997   
2  06-01-2015  63.020000  67.930000  3749400.0  67.989998  64.669998   
3  07-01-2015  63.509998  65.010002  2122600.0  66.760002  66.650002   
4  08-01-2015  67.129997  67.949997  2510300.0  69.690002  67.690002   

   Adjusted Close company_id  
0       70.400002       QRVO  
1       67.629997       QRVO  
2       64.669998       QRVO  
3       66.650002       QRVO  
4       67.690002       QRVO  


2397141

# Company Size Top Ten

In [ ]:
# limit the df by size to speed up computation
# get volume * open
# get avg of that by company, make a new dataframe with only one row per company
# plot x-axis as company name and "avg size" as y-axis

In [ ]:
top_ten_df = df.query('Open > 100')
top_ten_df.drop(columns=['Date', 'Adjusted Close', 'Low', 'High', 'Close'], inplace=True)
top_ten_df['size'] = top_ten_df.apply(lambda x: round((x.Volume * x.Open) / 1000, 2), axis=1)
top_ten_df.head()
top_ten_df['company_id'].unique()

In [72]:
ids = {'QRVO':0, 'RCL':0, 'RE':0, 'REGN':0, 'RHI':0, 'RIBT':0, 'RJF':0, 'RL':0, 'RLI':0, 'RMD':0, 'ROK':0, 'ROP':0, 'ROST':0, 'RSG':0, 'SBUX':0, 'SHW':0, 'SIVB':0, 'SLB':0,
'SLG':0, 'SNPS':0, 'SPG':0, 'SRE':0, 'STT':0, 'STX':0, 'STZ-B':0, 'SWK':0, 'SWKS':0, 'SYK':0, 'TAP':0, 'TEL':0, 'TMO':0, 'TMUS':0, 'TROW':0, 'TRV':0, 'TSCO':0, 'TTWO':0,'TW':0, 'TXN':0, 'TYL':0, 'UHS':0, 'ULTA':0, 'UNP':0, 'UPS':0, 'URI':0, 'V':0, 'VMC':0,
'VRSK':0, 'VRSN':0, 'WAT':0, 'WDC':0, 'WEC':0, 'WHR':0, 'WM':0, 'WSPOF':0, 'WST':0, 'WYNN':0, 'XOM':0, 'XYL':0, 'YUM':0, 'ZBH':0, 'ZTS':0, 'A':0, 'AAP':0, 'AAPL':0, 'ABBV':0, 'ABC':0, 'ABMD':0, 'ABT':0, 'ACN':0, 'ADI':0, 'ADP':0, 'ADSK':0, 'AEP':0, 'AIZ':0, 'AJG':0, 'AKAM':0, 'ALB':0, 'ALGN':0, 'ALLE':0, 'AMAT':0, 'AMD':0, 'AME':0,
'AMGN':0, 'AMP':0, 'AMT':0, 'AMZN':0, 'ANET':0, 'AON':0, 'APA':0, 'APD':0, 'ARE':0, 'ATVI':0, 'AVB':0, 'AVY':0, 'AWK':0, 'AXP':0, 'AZO':0, 'BA':0, 'BBY':0, 'BDX':0,
'BIIB':0, 'BIO':0, 'BLK':0, 'BR':0, 'BRK-A':0, 'BXP':0, 'C':0, 'CAT':0, 'CB':0, 'CCI':0, 'CDE':0, 'CDNS':0, 'CF':0, 'CHD':0, 'CHRW':0, 'CHTR':0, 'CINF':0, 'CLX':0, 'CME':0, 'CMG':0, 'CMI':0, 'COO':0, 'COP':0, 'COST':0, 'CRM':0, 'CTAS':0, 'DE':0,
'DFS':0, 'DG':0, 'DGX':0, 'DHI':0, 'DIS':0, 'DLTR':0, 'DOV':0, 'DPZ':0, 'DRI':0, 'DTE':0, 'DVA':0, 'DXCM':0, 'EA':0, 'ECL':0, 'ED':0, 'EFX':0, 'EL':0, 'EMN':0, 'EMR':0, 'ENS':0, 'EOG':0, 'EQIX':0, 'ESS':0, 'EW':0, 'EXR':0, 'FANG':0, 'FBHS':0, 'FDX':0, 'FFIV':0, 'FIS':0, 'FISV':0, 'FLT':0, 'FMC':0, 'FN':0, 'FRC':0, 'FRT':0, 'GD':0,
'GE':0, 'GILD':0, 'GOOG':0, 'GPC':0, 'GPN':0, 'GRMN':0, 'GWW':0, 'HAS':0, 'HCA':0, 'HD':0, 'HES':0, 'HII':0, 'HLT':0, 'HON':0, 'HSY':0, 'HUM':0, 'IBM':0, 'ICE':0, 'IDXX':0, 'IEX':0, 'IFF':0, 'ILMN':0, 'INTH':0, 'INTU':0, 'IPGP':0, 'ISRG':0, 'IT':0,
'ITW':0, 'JBHT':0, 'JCI':0, 'JKHY':0, 'JNJ':0, 'JNPR':0, 'JPM':0, 'KEYS':0, 'KMB':0, 'KMX':0, 'LDOS':0, 'LH':0, 'LMT':0, 'LOW':0, 'LRCX':0, 'LVS':0, 'LYB':0, 'LYV':0, 'MAA':0, 'MAR':0, 'MCD':0, 'MCK':0, 'MCO':0, 'MDT':0, 'MHK':0, 'MKTX':0, 'MLM':0, 'MMC':0, 'MMM':0, 'MNST':0, 'MOS':0, 'MPC':0, 'MRK':0, 'MSCI':0, 'MSFT':0, 'MSI':0,
'NFLX':0, 'NOC':0, 'NOW':0, 'NOXL':0, 'NSC':0, 'NTAP':0, 'NTRA':0, 'NTRR':0, 'NTRS':0, 'NVR':0, 'NVRO':0, 'ODFL':0, 'ORLY':0, 'OXY':0, 'PAYX':0, 'PCAR':0, 'PEP':0, 'PG':0, 'PH':0, 'PKG':0, 'PKI':0, 'PLD':0, 'PM':0, 'PNW':0, 'PPG':0, 'PRU':0,'PSX':0, 'PVH':0, 'PWR':0, 'PXD':0}

# top_ten_df.set_index("company_id", inplace=True)


for id, num in ids.items():
    new_df = top_ten_df.loc[id, 'size']
    new_df = pd.DataFrame(new_df)
    avg = new_df['size'].mean()
    ids[id] = avg

new_dict = {'id':[], 'size':[]}

for id, num in ids.items():
    new_dict['id'].append(id)
    new_dict['size'].append(num)

size_df = pd.DataFrame.from_dict(new_dict)

size_df['company'] = size_df['id'].apply(get_company_name)


size_df.drop(columns=['id'])

# REMOVE EVERYTHING UNDER 1 MILLION 5 HUNDRED THOUSAND, "SIVB" DOESNT WORK BUT I THINK ITS SMALL


HTTPError: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SIVB?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true

In [64]:
size_chart = alt.Chart(size_df).mark_bar().encode(
    x = alt.X('id', sort='-y'),
    y = alt.Y('size')
)
size_chart

alt.Chart(...)